In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
file_names=[]
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        #print(os.path.join(dirname, filename))
        file_names.append(os.path.join(dirname,filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
for i in file_names:
    if ".jpg" not in i:
        print(i)

/kaggle/input/target-hotspot-dataset/Target-hotspotDataset/valid/_annotations.csv
/kaggle/input/target-hotspot-dataset/Target-hotspotDataset/test/_annotations.csv
/kaggle/input/target-hotspot-dataset/Target-hotspotDataset/train/_annotations.csv
/kaggle/input/efficient_net/tensorflow2/default/1/efficientdet_d1_coco17_tpu-32/pipeline.config
/kaggle/input/efficient_net/tensorflow2/default/1/efficientdet_d1_coco17_tpu-32/saved_model/saved_model.pb
/kaggle/input/efficient_net/tensorflow2/default/1/efficientdet_d1_coco17_tpu-32/saved_model/variables/variables.index
/kaggle/input/efficient_net/tensorflow2/default/1/efficientdet_d1_coco17_tpu-32/saved_model/variables/variables.data-00000-of-00001
/kaggle/input/efficient_net/tensorflow2/default/1/efficientdet_d1_coco17_tpu-32/checkpoint/checkpoint
/kaggle/input/efficient_net/tensorflow2/default/1/efficientdet_d1_coco17_tpu-32/checkpoint/ckpt-0.index
/kaggle/input/efficient_net/tensorflow2/default/1/efficientdet_d1_coco17_tpu-32/checkpoint/ckpt-

DATA PREPROCESSING LAYER 


In [3]:
#now lets read the csv files 
import pandas as pd

valid_pd=pd.read_csv("/kaggle/input/target-hotspot-dataset/Target-hotspotDataset/valid/_annotations.csv")

test_pd=pd.read_csv("/kaggle/input/target-hotspot-dataset/Target-hotspotDataset/test/_annotations.csv")

train_pd=pd.read_csv("/kaggle/input/target-hotspot-dataset/Target-hotspotDataset/train/_annotations.csv")

In [4]:
train_pd

,filename,width,height,class,xmin,ymin,xmax,ymax
0,target234_jpeg.rf.8095c708f60bd118c9788393d457...,640,640,Target,138,93,549,524
1,sheared_hotspot193_jpeg.rf.7e7eb920017d4ceb7cc...,640,640,Hotpot,198,104,495,428
2,sheared_hotspot59_jpeg.rf.7ff838aeae45ad335d0f...,640,640,Hotpot,133,196,439,547
3,target168_jpeg.rf.832393e28d976b26e326a0dd0459...,640,640,Target,103,65,484,459
4,sheared_hotspot153_jpeg.rf.80cff7bf8efc2cc6a1f...,640,640,Hotpot,232,124,505,463
...,...,...,...,...,...,...,...,...
688,sheared_hotspot33_jpeg.rf.fee624d4b4b972d44070...,640,640,Hotpot,83,139,534,613
689,target160_jpeg.rf.ff87ff2d1c8c4d4aeb3aeb4c1368...,640,640,Target,69,78,472,522
690,target212_jpeg.rf.ff4259afbfb039e11669da32c40e...,640,640,Target,93,52,485,469
691,target134_jpeg.rf.f616992a05ca7c96b4944cd96001...,640,640,Target,96,154,497,596


In [5]:
#the object detection module has two parts the object classification module and the object localization module
import copy
changed_train_pd=train_pd.copy()

changed_train_pd

,filename,width,height,class,xmin,ymin,xmax,ymax
0,target234_jpeg.rf.8095c708f60bd118c9788393d457...,640,640,Target,138,93,549,524
1,sheared_hotspot193_jpeg.rf.7e7eb920017d4ceb7cc...,640,640,Hotpot,198,104,495,428
2,sheared_hotspot59_jpeg.rf.7ff838aeae45ad335d0f...,640,640,Hotpot,133,196,439,547
3,target168_jpeg.rf.832393e28d976b26e326a0dd0459...,640,640,Target,103,65,484,459
4,sheared_hotspot153_jpeg.rf.80cff7bf8efc2cc6a1f...,640,640,Hotpot,232,124,505,463
...,...,...,...,...,...,...,...,...
688,sheared_hotspot33_jpeg.rf.fee624d4b4b972d44070...,640,640,Hotpot,83,139,534,613
689,target160_jpeg.rf.ff87ff2d1c8c4d4aeb3aeb4c1368...,640,640,Target,69,78,472,522
690,target212_jpeg.rf.ff4259afbfb039e11669da32c40e...,640,640,Target,93,52,485,469
691,target134_jpeg.rf.f616992a05ca7c96b4944cd96001...,640,640,Target,96,154,497,596


In [6]:
print(changed_train_pd['filename'][1])

sheared_hotspot193_jpeg.rf.7e7eb920017d4ceb7ccd8ebdcfb1a102.jpg


In [7]:
#now first of all we change the class  to the categorical values 
#target----------->>>>>>. 0
#hotspot---------->>>>>>  1

changed_train_pd['class']=changed_train_pd['class'].replace('Target',0)
changed_train_pd['class']=changed_train_pd['class'].replace('Hotpot',1)

/tmp/ipykernel_33/1653966321.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  changed_train_pd['class']=changed_train_pd['class'].replace('Hotpot',1)


In [8]:
import matplotlib.pyplot as plt
def image_train(image_list,final_list):
    for i in range(len(image_list)):
        z=image_list[i]
        final_z="/kaggle/input/target-hotspot-dataset/Target-hotspotDataset/train/"+z
        l=plt.imread(final_z)
        final_list.append(l)
        
        
def image_test(image_list,final_list):
    for i in range(len(image_list)):
        z=image_list[i]
        final_z="/kaggle/input/target-hotspot-dataset/Target-hotspotDataset/test/"+z
        l=plt.imread(final_z)
        final_list.append(l)

NEW DATASET.CSV

In [9]:
changed_train_pd

,filename,width,height,class,xmin,ymin,xmax,ymax
0,target234_jpeg.rf.8095c708f60bd118c9788393d457...,640,640,0,138,93,549,524
1,sheared_hotspot193_jpeg.rf.7e7eb920017d4ceb7cc...,640,640,1,198,104,495,428
2,sheared_hotspot59_jpeg.rf.7ff838aeae45ad335d0f...,640,640,1,133,196,439,547
3,target168_jpeg.rf.832393e28d976b26e326a0dd0459...,640,640,0,103,65,484,459
4,sheared_hotspot153_jpeg.rf.80cff7bf8efc2cc6a1f...,640,640,1,232,124,505,463
...,...,...,...,...,...,...,...,...
688,sheared_hotspot33_jpeg.rf.fee624d4b4b972d44070...,640,640,1,83,139,534,613
689,target160_jpeg.rf.ff87ff2d1c8c4d4aeb3aeb4c1368...,640,640,0,69,78,472,522
690,target212_jpeg.rf.ff4259afbfb039e11669da32c40e...,640,640,0,93,52,485,469
691,target134_jpeg.rf.f616992a05ca7c96b4944cd96001...,640,640,0,96,154,497,596


In [10]:
print(changed_train_pd['filename'][0])

target234_jpeg.rf.8095c708f60bd118c9788393d457a1e0.jpg


SPLIT INTO TRAIN SET 

In [11]:
#here we first try to create the y_train and the x_train set
y_train=[]
for i in range(len(changed_train_pd)):
    if changed_train_pd['class'][i]==0:
        y_train.append([1,0])
    else:
        y_train.append([0,1])
        
x_train=[]
image_train(changed_train_pd['filename'],x_train)

print(len(x_train))
#now we typecast them into the numpy  array 
y_train=np.asarray(y_train)
x_train=np.asarray(x_train)


693


MODEL CREATION FOR CLASSIFICATION

In [12]:
#now first we create  deep learning model for classification
#we import the necessary dependencies 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Input,Dense,Conv2D,MaxPool2D,BatchNormalization,Dropout,Flatten

from keras.utils import plot_model 
from tensorflow.keras.models import Model


2024-09-20 03:50:46.448761: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-20 03:50:46.448920: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-20 03:50:46.604508: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
#here we create a neural network for classification module 
input_layer=Input(shape=(640,640,3))
conv_1_layer=Conv2D(64,3,3,activation='relu',padding='same')(input_layer)
Maxpool1=MaxPool2D()(conv_1_layer)
batchnormalization=BatchNormalization()(Maxpool1)
conv_2_layer=Conv2D(32,3,3,activation='relu',padding='same')(batchnormalization)
batchnormalization1=BatchNormalization()(conv_2_layer)
conv_3_layer=Conv2D(16,3,3,activation='relu',padding='same')(batchnormalization1)
batchnormalization2=BatchNormalization()(conv_3_layer)
conv_4_layer=Conv2D(8,3,3,activation='relu',padding='same')(batchnormalization2)
conv_5_layer=Conv2D(4,3,3,activation='relu',padding='same')(conv_4_layer)
conv_6_layer=Conv2D(1,3,3,activation='relu',padding='same')(conv_5_layer)
flatten_layer=Flatten()(conv_6_layer)
#now lets introduce the dropout layer 
dense1=Dense(2500,activation='relu')(flatten_layer)
dense2=Dense(1024,activation='relu')(dense1)
dropout1=Dropout(0.2)(dense2)
dense3=Dense(512,activation='relu')(dropout1)
dropout2=Dropout(0.3)(dense3)
dense4=Dense(256,activation='relu')(dense3)
dense5=Dense(128,activation='relu')(dense4)
dropout3=Dropout(0.1)(dense5)
dense6=Dense(10,activation='relu')(dropout3)
dense8=Dense(2,activation='softmax')(dense6)


#NOW WE TRY TO WRAP  THE MODEL 
classification_model=Model(inputs=input_layer,outputs=dense8)






In [ ]:
classification_model.summary()

In [ ]:
plot_model(classification_model,show_layer_names=True,show_shapes=True)


COMPILE THE MODEL , AND THEN  FIT THE MODEL 


In [ ]:
classification_model.compile(loss='binary_crossentropy',metrics=['accuracy'],
                            optimizer='adam')

FIT THE DATA INTO THE MODEL 

In [ ]:
classification_model.fit(x_train,y_train,batch_size=15,verbose=1,epochs=5)

TEST USING THE TEST DATASET

In [13]:
#now we try to get the test dataset
testing_pd=test_pd

testing_pd

,filename,width,height,class,xmin,ymin,xmax,ymax
0,sheared_hotspot128_jpeg.rf.06728f05688eb080511...,640,640,Hotpot,206,235,470,539
1,sheared_hotspot244_jpeg.rf.07a973dec627c2b80f9...,640,640,Hotpot,250,233,510,553
2,sheared_hotspot100_jpeg.rf.119f062b0cfc8eaed8e...,640,640,Hotpot,95,209,390,544
3,target161_jpeg.rf.07ce2223222352be95e94b00504c...,640,640,Target,1,24,500,530
4,target42_jpeg.rf.1a8e7b5cee712223533425b066ab0...,640,640,Target,136,131,513,550
...,...,...,...,...,...,...,...,...
155,target247_jpeg.rf.fc2e8967a3866eb2c4d9adb8f887...,640,640,Target,56,145,481,580
156,sheared_hotspot41_jpeg.rf.ee1205810a28bacd7f47...,640,640,Hotpot,144,99,415,432
157,sheared_hotspot234_jpeg.rf.f13785509d24d830d35...,640,640,Hotpot,214,92,492,455
158,sheared_hotspot204_jpeg.rf.f913ddfaabcde91b463...,640,640,Hotpot,137,178,449,508


In [14]:
testing_pd['class']=testing_pd['class'].replace('Hotpot',1)
testing_pd['class']=testing_pd['class'].replace('Target',0)


testing_pd

/tmp/ipykernel_34/755590333.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  testing_pd['class']=testing_pd['class'].replace('Target',0)


,filename,width,height,class,xmin,ymin,xmax,ymax
0,sheared_hotspot128_jpeg.rf.06728f05688eb080511...,640,640,1,206,235,470,539
1,sheared_hotspot244_jpeg.rf.07a973dec627c2b80f9...,640,640,1,250,233,510,553
2,sheared_hotspot100_jpeg.rf.119f062b0cfc8eaed8e...,640,640,1,95,209,390,544
3,target161_jpeg.rf.07ce2223222352be95e94b00504c...,640,640,0,1,24,500,530
4,target42_jpeg.rf.1a8e7b5cee712223533425b066ab0...,640,640,0,136,131,513,550
...,...,...,...,...,...,...,...,...
155,target247_jpeg.rf.fc2e8967a3866eb2c4d9adb8f887...,640,640,0,56,145,481,580
156,sheared_hotspot41_jpeg.rf.ee1205810a28bacd7f47...,640,640,1,144,99,415,432
157,sheared_hotspot234_jpeg.rf.f13785509d24d830d35...,640,640,1,214,92,492,455
158,sheared_hotspot204_jpeg.rf.f913ddfaabcde91b463...,640,640,1,137,178,449,508


HERE WE PREPARE THE TESTING LABELS 


In [15]:
y_test=[]
for i in range(len(testing_pd)):
    if testing_pd['class'][i]==0:
        y_test.append([1,0])
    else:
        y_test.append([0,1])

x_test=[]

image_test(testing_pd['filename'],x_test)

#now we typecast them into the numpy arrays
x_test=np.asarray(x_test)
y_test=np.asarray(y_test)



EVALUATING THE MODEL 

In [ ]:
loss,accuracy=classification_model.evaluate(x_test,y_test,verbose=1,batch_size=10)
print(loss)
print(accuracy)

NOW LETS SEE. SOME PREDICTIONS OF THE MODEL 

In [ ]:
detected=[]
for i in range(len(x_test)):
    l=x_test[i]
    expanded_l=np.expand_dims(l,axis=0)
    z=classification_model.predict(expanded_l)
    max_index=np.argmax(z)
    detected.append(max_index)
     #this gives us the probabilities 
    #choose the max among above 
print(detected)


In [ ]:
y_test_comparing=testing_pd['class']
sum1=0
for j in range(len(y_test_comparing)):
    if y_test_comparing[j]==detected[j]:
        sum1=sum1+1
print(sum1/len(y_test_comparing))
        
    

In [18]:
def image_valid(image_list,final_list):
    for i in range(len(image_list)):
        z=image_list[i]
        final_z="/kaggle/input/target-hotspot-dataset/Target-hotspotDataset/valid/"+z
        l=plt.imread(final_z)
        final_list.append(l)

In [19]:
valid_pd=valid_pd.replace('Hotpot',1)
valid_pd=valid_pd.replace('Target',0)

/tmp/ipykernel_34/635522676.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  valid_pd=valid_pd.replace('Target',0)


In [20]:
#now lets check for the performance into valid dataset 
x_valid=[]
image_valid(valid_pd['filename'],x_valid)

y_valid=valid_pd['class']



In [21]:
print(len(y_valid))

154


In [22]:
print(len(x_valid))

154


In [ ]:
#now lets predict the values 
detected1=[]
for i in range(len(x_valid)):
    l=x_valid[i]
    expanded_l=np.expand_dims(l,axis=0)
    z=classification_model.predict(expanded_l)
    max_index=np.argmax(z)
    detected1.append(max_index)
     #this gives us the probabilities 
    #choose the max among above 
print(len(detected1))


In [ ]:
sum2=0
for i in range(len(detected1)):
    if detected1[i]==y_valid[i]:
        sum2=sum2+1
        
print(sum2/len(detected1))
        
        
        

In [ ]:
#now lets save the model 

classification_model.save("/kaggle/working/classification_model.h5")
classification_model.save_weights("/kaggle/working/classification_model_saved.weights.h5")


The above model can be be made smaller using the tensorflow lite 

In [ ]:
import tensorflow as tf

# Load the saved Keras model (this includes the model structure and weights)
model = tf.keras.models.load_model('/kaggle/working/classification_model.h5')

# Create a TensorFlow Lite Converter from the Keras model
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# Optional: Apply optimizations if needed
# For example, applying post-training quantization
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Convert the model to TensorFlow Lite format
tflite_model = converter.convert()

# Save the converted TensorFlow Lite model to a .tflite file
with open('/kaggle/working/classification_model.tflite', 'wb') as f:
    f.write(tflite_model)


NOW. HERE. WE TRY TO MAKE THE IMAGE LOCALIZATION MODULE 

What we propose here is we have the bounding box in the data set using  the autoencoder , we can extraxt the feature of the hotspot and the target over all the available samples , and then we search those feature vectors


In [31]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.models import Model

# Load the base model
base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

# Add custom layers
x = base_model.output
flatten_layer = Flatten()(x)
x1 = Dense(1024, activation='relu')(flatten_layer)
x2 = Dense(4, activation='relu')(x1)

# Output layer for bounding box coordinates
bbox_output = Dense(4, activation='sigmoid', name='bbox_output')(x2)
model = Model(inputs=base_model.input, outputs=bbox_output)

# IoU function
def iou(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    
    x1 = tf.maximum(y_true[..., 0], y_pred[..., 0])
    y1 = tf.maximum(y_true[..., 1], y_pred[..., 1])
    x2 = tf.minimum(y_true[..., 2], y_pred[..., 2])
    y2 = tf.minimum(y_true[..., 3], y_pred[..., 3])

    intersection = tf.maximum(x2 - x1, 0) * tf.maximum(y2 - y1, 0)

    area_true = (y_true[..., 2] - y_true[..., 0]) * (y_true[..., 3] - y_true[..., 1])
    area_pred = (y_pred[..., 2] - y_pred[..., 0]) * (y_pred[..., 3] - y_pred[..., 1])

    union = area_true + area_pred - intersection
    iou = intersection / tf.maximum(union, 1e-6)
    return iou

# Custom loss function combining MSE and IoU loss
def combined_loss(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    
    mse_loss = tf.reduce_mean(tf.square(y_true - y_pred))
    iou_loss_value = 1 - iou(y_true, y_pred)
    return mse_loss + iou_loss_value

# IoU metric
def iou_metric(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    
    return tf.reduce_mean(iou(y_true, y_pred))

# Compile the model
model.compile(optimizer='adam', loss=combined_loss, metrics=[iou_metric])
model.summary()


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer_2[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 66,488,408 (253.63 MB)

 Trainable params: 66,454,296 (253.50 MB)

 Non-trainable params: 34,112 (133.25 KB)

In [32]:
import cv2
def reduce_image_size(img_path_list,image_array):
    for i in range(len(img_path_list)):
        z=img_path_list[i]
        final_z="/kaggle/input/target-hotspot-dataset/Target-hotspotDataset/train/"+z
        image=cv2.imread(final_z)
        reshaped_image=cv2.resize(image,(224,224))
        
        image_array.append(reshaped_image)
    
def reduce_image(img_path):
    
    z=img_path
    final_z="/kaggle/input/target-hotspot-dataset/Target-hotspotDataset/test/"+z
    image=cv2.imread(final_z)
    reshaped_image=cv2.resize(image,(224,224))
    return reshaped_image

In [33]:
#now lets create the bounding box dataset
bounding_box_coordinates=[]
for i in range(len(changed_train_pd)):
    l=[]
    l.append(changed_train_pd['xmin'][i])
    l.append(changed_train_pd['ymin'][i])
    l.append(changed_train_pd['xmax'][i])
    l.append(changed_train_pd['ymax'][i])
    bounding_box_coordinates.append(l)
    
bounding_box_coordinates=np.asarray(bounding_box_coordinates)

#now lets fit the data
x_train_reshaped_image=[]
reduce_image_size(changed_train_pd['filename'],x_train_reshaped_image)

x_train_reshaped_image=np.asarray(x_train_reshaped_image)

#now lets fit the data to the model 
model.fit(x_train_reshaped_image,bounding_box_coordinates,epochs=10,batch_size=15,verbose=1)




Epoch 1/10
 2/47 ━━━━━━━━━━━━━━━━━━━━ 4s 93ms/step - iou_metric: 0.0000e+00 - loss: 139305.7188  

W0000 00:00:1724383875.205912     112 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


47/47 ━━━━━━━━━━━━━━━━━━━━ 68s 476ms/step - iou_metric: 0.0000e+00 - loss: 141210.3438
Epoch 2/10
 1/47 ━━━━━━━━━━━━━━━━━━━━ 5s 115ms/step - iou_metric: 0.0000e+00 - loss: 137541.7656

W0000 00:00:1724383897.149174     115 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


47/47 ━━━━━━━━━━━━━━━━━━━━ 3s 69ms/step - iou_metric: 0.0000e+00 - loss: 141557.8594
Epoch 3/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 3s 69ms/step - iou_metric: 0.0000e+00 - loss: 141069.9688
Epoch 4/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 3s 69ms/step - iou_metric: 0.0000e+00 - loss: 142603.0156
Epoch 5/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 3s 68ms/step - iou_metric: 0.0000e+00 - loss: 141974.6875
Epoch 6/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 3s 69ms/step - iou_metric: 0.0000e+00 - loss: 143065.7812
Epoch 7/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 3s 70ms/step - iou_metric: 0.0000e+00 - loss: 142075.7812
Epoch 8/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 3s 69ms/step - iou_metric: 0.0000e+00 - loss: 142171.4688
Epoch 9/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 3s 69ms/step - iou_metric: 0.0000e+00 - loss: 143562.4375
Epoch 10/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 3s 69ms/step - iou_metric: 0.0000e+00 - loss: 143449.1875


In [34]:
#lets predict some values 
z=test_pd['xmin'][1]
z1=test_pd['ymin'][1]
z2=test_pd['xmax'][1]
z3=test_pd['ymax'][1]

#now lets predict for one x_test
z=test_pd['filename'][1]
final_z="/kaggle/input/target-hotspot-dataset/Target-hotspotDataset/test/"+z
image=cv2.imread(final_z)

print(image.shape)
#now lets increase its dimension

image=reduce_image(z)
print(image.shape)

image=np.expand_dims(image,axis=0)
print(image.shape)

#now lets predict the bounding box_coordinates
l=model.predict(image)

print(l)

(640, 640, 3)
(224, 224, 3)
(1, 224, 224, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
[[1. 0. 0. 1.]]


HERE WE WILL TRY BY USING THE YOLO V8 CUSTOM DATA TRAINED 


In [ ]:
#FOR TRAINING WE FIRST NEED TO NORMALIZE THE COORDINATES 


  IMAGE BASE  LOCALIZATION 

In [17]:
changed_train_pd

,filename,width,height,class,xmin,ymin,xmax,ymax
0,target234_jpeg.rf.8095c708f60bd118c9788393d457...,640,640,0,138,93,549,524
1,sheared_hotspot193_jpeg.rf.7e7eb920017d4ceb7cc...,640,640,1,198,104,495,428
2,sheared_hotspot59_jpeg.rf.7ff838aeae45ad335d0f...,640,640,1,133,196,439,547
3,target168_jpeg.rf.832393e28d976b26e326a0dd0459...,640,640,0,103,65,484,459
4,sheared_hotspot153_jpeg.rf.80cff7bf8efc2cc6a1f...,640,640,1,232,124,505,463
...,...,...,...,...,...,...,...,...
688,sheared_hotspot33_jpeg.rf.fee624d4b4b972d44070...,640,640,1,83,139,534,613
689,target160_jpeg.rf.ff87ff2d1c8c4d4aeb3aeb4c1368...,640,640,0,69,78,472,522
690,target212_jpeg.rf.ff4259afbfb039e11669da32c40e...,640,640,0,93,52,485,469
691,target134_jpeg.rf.f616992a05ca7c96b4944cd96001...,640,640,0,96,154,497,596


In [27]:
import matplotlib.pyplot as plt
def change_image_to_specialize_area(changed_train_pd,changed_image_array):
    image_path="/kaggle/input/target-hotspot-dataset/Target-hotspotDataset/train/"
    for i in range(len(changed_train_pd)):
        image_path_added=image_path+changed_train_pd['filename'][i]
        z=plt.imread(image_path_added)
        xmin=changed_train_pd['xmin'][i]
        ymin=changed_train_pd['ymin'][i]
        xmax=changed_train_pd['xmax'][i]
        ymax=changed_train_pd['ymax'][i]
        
        area_of_interest=z[xmin:xmax,ymin:ymax]
        changed_image_array.append(area_of_interest)
    return changed_image_array
        
        
        

In [28]:
changed_image_array=[]
changed_image_array=change_image_to_specialize_area(changed_train_pd,changed_image_array)

print(len(changed_image_array))


693


In [34]:
#let find the average of the shape 
sum1=0
sum2=0
for i in range(len(changed_image_array)):
    l,b,h=changed_image_array[i].shape
    sum1=sum1+l
    sum2=sum2+b
    
sum1=sum1//len(changed_image_array)
sum2=sum2//len(changed_image_array)
print(sum1,sum2,3)
    
    
    

364 398 3


In [42]:
from skimage import transform

def resized_image_interest_area(changed_image_array):
    for i in range(len(changed_image_array)):
        changed_image_array[i] = transform.resize(changed_image_array[i], (100, 100))

# Example usage:
# resized_image_interest_area(changed_image_array)



    
resized_image_interest_area(changed_image_array)

print(changed_image_array[0].shape)


(100, 100, 3)


In [39]:
#now lets make the neural network , this makes the images into differnet scales and then blurs the image , to get various localized model 
#import the dependencies 

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers 
from tensorflow.keras.layers import *
from keras.utils import plot_model 
from keras.models import Model 


In [41]:
input1=Input((324,324,3))

conv2=Conv2D(10,(3,3),activation='relu',padding='same')(input1)
conv3=Conv2DTranspose(10,(3,3),strides=2,activation='relu',padding='same')(conv2)

model=Model(inputs=input1,outputs=conv3)
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 324, 324, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 324, 324, 10)   │           280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_1              │ (None, 648, 648, 10)   │           910 │
│ (Conv2DTranspose)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,190 (4.65 KB)

 Trainable params: 1,190 (4.65 KB)

 Non-trainable params: 0 (0.00 B)

In [9]:
class Essential_localization(tf.keras.Model):
    def __init__(self):
        self.model=None
        self.activation='relu'
        self.input_size=(100,100,3)
        
    def upsample_spatial_module(self,input_tensor):
        #this upsampling module basically increases the sizes to 6 different levels 
        output_tensor1=Conv2DTranspose(3,(3,3),strides=1,activation='relu')(input_tensor)
        output_tensor2=Conv2DTranspose(3,(5,5),strides=1,activation='relu')(output_tensor1)
        
        output_tensor3=Conv2DTranspose(3,(3,3),strides=2,activation='relu')(output_tensor2)
        output_tensor4=Conv2DTranspose(3,(5,5),strides=2,activation=='relu')(output_tensor3)
        
        output_tensor5=Conv2DTranspose(3,(3,3),strides=4,activation='relu')(output_tensor4)
        output_tensor6=Conv2DTranspose(3,(5,5),strides=4,activation=='relu')(output_tensor5)
        
        
        output_tensor7=Conv2DTranspose(3,(3,3),strides=8,activation='relu')(output_tensor6)
        output_tensor8=Conv2DTranspose(3,(5,5),strides=8,activation=='relu')(output_tensor7)
        
        #lets return the tensor we get from each layer , so that we proceed towards the next part of blurring them 
        return [output_tensor1,output_tensor2,output_tensor3,output_tensor4,output_tensor5,output_tensor6,output_tensor7,output_tensor8]
    
    def blurring_module(self,blurring_tensors):
        #here we add the nise of the Guassian nature
        for i in range(len(blurring_tensors)):
            z=blurring_tensors[i]
            noise=tf.shape.normal(shape=tf.shape(blurring_tensors[i]),mean=0.0,stddev=0.2)
            blurring_tensors[i]=blurring_tensors[i]+noise #this adds the gaussian noise to the tensors
            
        return blurring_tensors
    
    def feature_subtraction(self,blurring_vector1,blurring_vector2,blurring_vector3,blurring_vector4,blurring_vector5,threshold):
        #for now we would fix a threshold 
        #first we typecast all the blurring tensors into a numpy array 
        blurring_vector1=np.asarray(blurring_vector1)
        blurring_vector2=np.asarray(blurring_vector2)
        blurring_vector3=np.asarray(blurring_vector3)
        blurring_vector4=np.asarray(blurring_vector4)
        blurring_vector5=np.asarray(blurring_vector5)
        
        blurring_vectors=[blurring_vector1,blurring_vector2,blurring_vector3,blurring_vector4,blurring_vetor5]
        ######## start from here ###################
        weight_vectors=[]
        for i in range(1,len(blurring_vectors)):
            weight_vector_per_level=[]
            for j in range(len(blurring_vectors[i])):
                z=blurring_vectors[0][j]
                z1=blurring_vectors[i][j]
                k=z-z1           
                #now  we apply the thresholding operation 
                length,breadth=k.shape
                weight_vector=np.zeros(length,breadth)
                for i1<length and j1<breadth:
                    if k[i1][j1]<threshold:
                        k[i1][j1]=0
                    else:
                        weight_vector[i1][j1]=n-i
                weight_vector_per_level.append(weight_vector)
                
            final_weight=np.zeros(length,breadth)
            for i2 in range(len(weight_vector_per_level)):
                final_weight =final_weight+weight_vector_per_level[i2]
                
            weight_vectors.append(final_weight)  #this has created the weight vector for the relative importance of the features 
            #for each of the 8 levels
            
            
        return weight_vectors
    
    def weight_feature_importance(self,weight_vectors,weight1,weight2,weight3,weight4,weight5,weight6,weight7,weight8):
        
        #each pair represent the same shape , each with different kernel size 
        #we again use the weighted approach for finding the weight of the final kernel size 
        #if the feature is found in the bigger kernel size we give it more importance ,then when found with the kernel size of 3x3
        #assign the weight2 more value than the weight1
        weight_filter=weight1*weight_vectors[0]+weight2*weight_vectors[1]
        weight_filter1=weight3*weight_vectors[2]+weight4*weight_vectors[3]
        weight_filter2=weight5*weight_vectors[4]+weight6*weight_vectors[5]
        weight_filter3=weight7*weight_vectors[6]+weight8*weight_vectors[7]
        
        return [weight_filter,weight_filter1,weight_filter2,weight_filter3]
    
    def print_the_feature_vectors(self,x):
        for i in range(len(x)):
            print(x.shape)
            
    def convert_into_tensors(self,x):
        tensor_list=[]
        for i in range(x):
            tensor=tf.convert_to_tensor(x[i])
            tensor_list.append(tensor)
        return tensor_list
            
        
        
    def create_model(self):
        input1=self.model.add(Input((self.input_size)))
        #after taking the input image size , we make it of different sizes 
        #using the scaling operation 
        #here we find the feature vectors 
        conv_1=self.model.add(Conv2D(3,(3,3),padding='same',activation='relu'))(input1)
        conv_2=self.model.add(Conv2D(1,(3,3),padding='same',activation='relu'))(conv_1)
        z=self.model.add(upsample_spatial_module(conv_2))
        
        #after getting the images of different shapes , we need to make it blurred by adding noise in it 
        
        z_blurred_1=blurring_module(z) 
        #this creates the level one blurring 
        
        z_blurred_2=blurring_module(z_blurred_1)
        #this creates level 2 blurring 
        
        z_blurred_3=blurring_module(z_blurred_2)
        #this creates level 3 blurring 
        
        z_blurred_4=blurring_module(z_blurred_3)
        #this creates level 4 blurring 
        
        z_blurred_5=blurring_module(z_blurred_4)
        #this creates level 4 blurring 
        #here we get the blurred images across different levels and at different intensities 
        
        #similar to sift algorithm we need to find the good features from the blurred vectors 
        
        #now we perform the feature subtraction 
        weight_vectors =feature_subtraction(blurring_vector1,blurring_vector2,blurring_vector3,blurring_vector4,blurring_vector5,10)
        
        weight_feature_importance=weight_feature_importance(weight_vectors,5,4,3,3,2,2,1,1)
        
        #now we get the features arrays with their weights 
        #now lets convert them into the tensors 
        tensor_list=convert_into_tensors(weight_feature_importance)
        #now we have the list of tensors at different levels 
        
        #
        
        
        
        
        
        
        
        


        
        
        
        
        
        
        
        
        
        
        


ValueError: A KerasTensor cannot be used as input to a TensorFlow function. A KerasTensor is a symbolic placeholder for a shape and dtype, used when constructing Keras Functional models or Keras Functions. You can only use it as input to a Keras layer or a Keras operation (from the namespaces `keras.layers` and `keras.operations`). You are likely doing something like:

```
x = Input(...)
...
tf_fn(x)  # Invalid.
```

What you should do instead is wrap `tf_fn` in a layer:

```
class MyLayer(Layer):
    def call(self, x):
        return tf_fn(x)

x = MyLayer()(x)
```


In [16]:
import tensorflow as tf

class Essential_localization(tf.keras.Model):
    def __init__(self):
        super(Essential_localization, self).__init__()
        self.activation = 'relu'
        self.input_size = (100, 100, 3)

    def upsample_spatial_module(self, input_tensor):
        output_tensor1 = Conv2DTranspose(3, (3, 3), strides=1,padding='same', activation='relu')(input_tensor)
        output_tensor2 = Conv2DTranspose(3, (5, 5), strides=1,padding='same', activation='relu')(output_tensor1)
        
        output_tensor3 = Conv2DTranspose(3, (3, 3), strides=1,padding='same', activation='relu')(output_tensor2)
        output_tensor4 = Conv2DTranspose(3, (5, 5), strides=1,padding='same', activation='relu')(output_tensor3)
        
        output_tensor5 = Conv2DTranspose(3, (3, 3), strides=1, padding='same',activation='relu')(output_tensor4)
        output_tensor6 = Conv2DTranspose(3, (5, 5), strides=1,padding='same', activation='relu')(output_tensor5)
        
        output_tensor7 = Conv2DTranspose(3, (3, 3), strides=1,padding='same', activation='relu')(output_tensor6)
        output_tensor8 = Conv2DTranspose(3, (5, 5), strides=1, padding='same',activation='relu')(output_tensor7)
        
        return [output_tensor1,output_tensor2,output_tensor3,output_tensor4,output_tensor5,output_tensor6,output_tensor7,output_tensor8]

    def blurring_module(self, blurring_tensors):
        blurred_tensors = []
        for tensor2 in blurring_tensors:
        # Ensure tensor2 is converted to a TensorFlow tensor
            tensor2_tf = tf.convert_to_tensor(tensor2)
        
            noise = tf.random.normal(shape=tf.shape(tensor2_tf), mean=0.0, stddev=0.2)
            blurred_tensors.append(tensor2_tf + 2 * noise)  # Adds Gaussian noise to the tensor
        return blurred_tensors

   

    def feature_subtraction(self,*blurred_tensors,threshold):
        #for now we would fix a threshold 
        
        ######## start from here ###################
        weight_vectors=[]
        for i in range(1,len(blurred_tensors)):
            weight_vector_per_level=[]
            blurring_vectors=blurred_tensors[i].numpy()
            for j in range(len(blurring_vectors[i])):
                z=blurring_vectors[0][j]
                z1=blurring_vectors[i][j]
                k=z-z1           
                #now  we apply the thresholding operation 
                length,breadth=k.shape
                weight_vector=np.zeros(length,breadth)
                for i1,j1 in i1<length and j1<breadth:
                    if k[i1][j1]<threshold:
                        k[i1][j1]=0
                    else:
                        weight_vector[i1][j1]=n-i
                weight_vector_per_level.append(weight_vector)
                
            final_weight=np.zeros(length,breadth)
            for i2 in range(len(weight_vector_per_level)):
                final_weight =final_weight+weight_vector_per_level[i2]
                
            weight_vectors.append(final_weight)  #this has created the weight vector for the relative importance of the features 
            #for each of the 8 levels
            
            
        return weight_vectors
    
    def weight_feature_importance(self,weight_vectors,weight1,weight2,weight3,weight4,weight5,weight6,weight7,weight8):
        
        #each pair represent the same shape , each with different kernel size 
        #we again use the weighted approach for finding the weight of the final kernel size 
        #if the feature is found in the bigger kernel size we give it more importance ,then when found with the kernel size of 3x3
        #assign the weight2 more value than the weight1
        weight_filter=weight1*weight_vectors[0]+weight2*weight_vectors[1]
        weight_filter1=weight3*weight_vectors[2]+weight4*weight_vectors[3]
        weight_filter2=weight5*weight_vectors[4]+weight6*weight_vectors[5]
        weight_filter3=weight7*weight_vectors[6]+weight8*weight_vectors[7]
        
        return [weight_filter,weight_filter1,weight_filter2,weight_filter3]
    
    def print_the_feature_vectors(self,x):
        for i in range(len(x)):
            print(x.shape)



    def call(self, input_image):
        input_image=Input((100,100,3))
        conv_1 = Conv2D(3, (3, 3), padding='same', activation='relu')(input_image)
        conv_2 =Conv2D(1, (3, 3), padding='same', activation='relu')(conv_1)
        
        z = self.upsample_spatial_module(conv_2)
        
        blurred_tensors0 = self.blurring_module(z)
        
        blurred_tensors1 = self.blurring_module(blurred_tensors0)
        blurred_tensors2 = self.blurring_module(blurred_tensors1)
        blurred_tensors3 = self.blurring_module(blurred_tensors2)
        
        blurred_tensors=[blurred_tensors0,blurred_tensors1,blurred_tensors2,blurred_tensors3]
        
        
        weight_vectors = self.feature_subtraction(*blurred_tensors)
        
        weight_feature_importance=self.weight_feature_importance(self,weight_vectors,4,3,3,3,2,2,1,1)
        
        return weight_feature_importance

# Example usage
input_image = tf.random.normal([1, 100, 100, 3])  # Dummy input
model = Essential_localization()
output_tensors = model.call(input_image)

# Print output tensor shapes
for tensor in output_tensors:
    print(tensor.shape)


ValueError: A KerasTensor cannot be used as input to a TensorFlow function. A KerasTensor is a symbolic placeholder for a shape and dtype, used when constructing Keras Functional models or Keras Functions. You can only use it as input to a Keras layer or a Keras operation (from the namespaces `keras.layers` and `keras.operations`). You are likely doing something like:

```
x = Input(...)
...
tf_fn(x)  # Invalid.
```

What you should do instead is wrap `tf_fn` in a layer:

```
class MyLayer(Layer):
    def call(self, x):
        return tf_fn(x)

x = MyLayer()(x)
```


In [25]:
file_names="/kaggle/input/target-hotspot-dataset/Target-hotspotDataset/train/"+changed_train_pd['filename'][0]
z=plt.imread(file_names)
print(z.shape)
z=z[changed_train_pd['xmin'][0]:changed_train_pd['xmax'][0],changed_train_pd['ymin'][0]:changed_train_pd['ymax'][0]]
print(z.shape)

(640, 640, 3)
(411, 431, 3)
